In [1]:
!pip install -q roboflow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 39.2 MB/s eta 0:00:00


In [2]:
import os
import shutil
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from roboflow import Roboflow
from sklearn.metrics import average_precision_score

# Parameters
API_KEY = ""#<--------------------------------- Replace this
img_size = (224, 224)
batch_size = 32
merged_dir = "/content/merged_dataset"
AUTOTUNE = tf.data.AUTOTUNE


In [3]:
from google.colab import userdata
my_api=userdata.get('my_api')

In [4]:
datasets_info = [
    ("zaghamshamsi", "soil-classification", 1),
    ("ai-usztq", "toprak-verim-analizi", 1),
    ("demian-a5xbv", "soil-analysis-55sc2", 1)
]

os.makedirs(merged_dir, exist_ok=True)
rf = Roboflow(api_key=my_api)

for workspace, project_name, version_number in datasets_info:
    project = rf.workspace(workspace).project(project_name)
    version = project.version(version_number)
    dataset = version.download("folder")

    for split in ["train", "valid", "test"]:
        split_path = os.path.join(dataset.location, split)
        if not os.path.exists(split_path):
            continue

        for class_folder in os.listdir(split_path):
            class_path = os.path.join(split_path, class_folder)
            if os.path.isdir(class_path):
                dest_class_path = os.path.join(merged_dir, class_folder)
                os.makedirs(dest_class_path, exist_ok=True)

                for file in os.listdir(class_path):
                    src_file = os.path.join(class_path, file)
                    dst_file = os.path.join(dest_class_path, f"{workspace}_{project_name}_{file}")
                    shutil.copy(src_file, dst_file)


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Soil-Classification-1 in folder:: 100%|██████████| 4306/4306 [00:01<00:00, 2220.33it/s]


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Toprak-verim-Analizi-1 in folder:: 100%|██████████| 1163/1163 [00:00<00:00, 2401.92it/s]


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Soil-Analysis-1 in folder:: 100%|██████████| 162/162 [00:00<00:00, 3645.21it/s]


In [5]:
classes_to_keep = [
    "Clay soil", "Black Soil", "Alluvial soil",
    "Red soil",  "loam", "sandy",
]

for class_name in os.listdir(merged_dir):
    if class_name not in classes_to_keep:
        shutil.rmtree(os.path.join(merged_dir, class_name))
        print(f"❌ Deleted: {class_name}")


❌ Deleted: Sandy loam
❌ Deleted: heavy
❌ Deleted: Clayey soils
❌ Deleted: medium
❌ Deleted: Laterite soil
❌ Deleted: Yellow Soil
❌ Deleted: Sandy soil
❌ Deleted: Loamy soil


In [6]:
dataset_path = "/content/merged_dataset"
img_size = (224, 224)
batch_size = 32
AUTOTUNE = tf.data.AUTOTUNE


In [7]:
raw_train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    seed=123,
    validation_split=0.2,
    subset="training",
    image_size=img_size,
    batch_size=batch_size
)

raw_val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    seed=123,
    validation_split=0.2,
    subset="validation",
    image_size=img_size,
    batch_size=batch_size
)

class_names = raw_train_ds.class_names
num_classes = len(class_names)

train_ds = raw_train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = raw_val_ds.prefetch(buffer_size=AUTOTUNE)


Found 3135 files belonging to 6 classes.
Using 2508 files for training.
Found 3135 files belonging to 6 classes.
Using 627 files for validation.


In [8]:
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
    layers.RandomBrightness(0.2)
])


In [9]:
base_model = tf.keras.applications.EfficientNetB0(
    input_shape=img_size + (3,),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False  # Freeze base model for feature extraction

model = keras.Sequential([
    keras.Input(shape=img_size + (3,)),
    data_augmentation,
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,214,313 (16.08 MB)

 Trainable params: 164,742 (643.52 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [10]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=25
)


Epoch 1/25
79/79 ━━━━━━━━━━━━━━━━━━━━ 31s 167ms/step - accuracy: 0.6804 - loss: 0.8199 - val_accuracy: 0.9250 - val_loss: 0.2320
Epoch 2/25
79/79 ━━━━━━━━━━━━━━━━━━━━ 12s 125ms/step - accuracy: 0.9014 - loss: 0.2687 - val_accuracy: 0.9490 - val_loss: 0.1898
Epoch 3/25
79/79 ━━━━━━━━━━━━━━━━━━━━ 9s 118ms/step - accuracy: 0.9237 - loss: 0.2205 - val_accuracy: 0.9362 - val_loss: 0.1985
Epoch 4/25
79/79 ━━━━━━━━━━━━━━━━━━━━ 10s 117ms/step - accuracy: 0.9370 - loss: 0.1734 - val_accuracy: 0.9649 - val_loss: 0.1663
Epoch 5/25
79/79 ━━━━━━━━━━━━━━━━━━━━ 11s 124ms/step - accuracy: 0.9429 - loss: 0.1515 - val_accuracy: 0.9490 - val_loss: 0.1599
Epoch 6/25
79/79 ━━━━━━━━━━━━━━━━━━━━ 10s 119ms/step - accuracy: 0.9636 - loss: 0.1239 - val_accuracy: 0.9474 - val_loss: 0.1580
Epoch 7/25
79/79 ━━━━━━━━━━━━━━━━━━━━ 10s 120ms/step - accuracy: 0.9441 - loss: 0.1556 - val_accuracy: 0.9426 - val_loss: 0.1878
Epoch 8/25
79/79 ━━━━━━━━━━━━━━━━━━━━ 11s 123ms/step - accuracy: 0.9603 - loss: 0.1284 - val_accur

In [11]:
y_true = []
y_pred = []

for images, labels in val_ds:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))

y_true = np.array(y_true)
y_pred = np.array(y_pred)

aps = []
for class_id in range(num_classes):
    y_true_binary = (y_true == class_id).astype(int)
    y_pred_binary = (y_pred == class_id).astype(int)
    ap = average_precision_score(y_true_binary, y_pred_binary)
    aps.append(ap)

mean_ap = np.mean(aps)
print(f"mAP: {mean_ap:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
mAP: 0.8549


In [27]:
base_model.trainable = True  # Unfreeze EfficientNet

model.compile(
    optimizer=keras.optimizers.Adam(1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history_finetune = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
)


Epoch 1/5
79/79 ━━━━━━━━━━━━━━━━━━━━ 78s 410ms/step - accuracy: 0.9706 - loss: 0.0814 - val_accuracy: 0.9282 - val_loss: 0.2105
Epoch 2/5
79/79 ━━━━━━━━━━━━━━━━━━━━ 30s 384ms/step - accuracy: 0.9752 - loss: 0.0821 - val_accuracy: 0.9426 - val_loss: 0.2026
Epoch 3/5
79/79 ━━━━━━━━━━━━━━━━━━━━ 29s 370ms/step - accuracy: 0.9548 - loss: 0.1281 - val_accuracy: 0.9458 - val_loss: 0.1984
Epoch 4/5
79/79 ━━━━━━━━━━━━━━━━━━━━ 41s 368ms/step - accuracy: 0.9710 - loss: 0.0922 - val_accuracy: 0.9506 - val_loss: 0.1949
Epoch 5/5
79/79 ━━━━━━━━━━━━━━━━━━━━ 41s 374ms/step - accuracy: 0.9741 - loss: 0.0789 - val_accuracy: 0.9506 - val_loss: 0.1914


In [28]:
y_true = []
y_pred = []

for images, labels in val_ds:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))

y_true = np.array(y_true)
y_pred = np.array(y_pred)

aps = []
for class_id in range(num_classes):
    y_true_binary = (y_true == class_id).astype(int)
    y_pred_binary = (y_pred == class_id).astype(int)
    ap = average_precision_score(y_true_binary, y_pred_binary)
    aps.append(ap)

mean_ap = np.mean(aps)
print(f"mAP after Fine-Tuning: {mean_ap:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
mAP after Fine-Tuning: 0.8521


In [29]:
model.save("efficientnet_model_soils.keras")